In [ ]:
from PIL import Image
import numpy as np
import hashlib
import os
import sys
import torch
import torch_xla.utils.tf_record_reader as tfrr

import tqdm as notebook_tqdm

In [ ]:
train_files = [
    'gs://imagenet-jt/train/train-00000-of-01024',
    # 'gs://imagenet-jt/train/train-00001-of-01024',
    # 'gs://imagenet-jt/train/train-00002-of-01024',
    # 'gs://imagenet-jt/train/train-00003-of-01024',
    # 'gs://imagenet-jt/train/train-00004-of-01024',
]

In [ ]:
def decode(ex):
    
    w = ex['image/width'].item()
    h = ex['image/height'].item()
    imgb = ex['image/encoded'].numpy().tobytes()

    # m = hashlib.md5()
    # m.update(imgb)
    # print('HASH = {}'.format(m.hexdigest()))

    image = Image.frombytes('RGB', (w, h), imgb, 'jpeg', 'RGB', None)
    
    npa = np.asarray(image)
    
    return torch.from_numpy(npa), image

In [ ]:
def readem(path):
    transforms = {
      'image/class/label': 'STR',
      # 'image/class/synset': 'STR',
      # 'image/format': 'STR',
      # 'image/class/text': 'STR',
      # 'image/colorspace': 'STR',
    }
    r = tfrr.TfRecordReader(path)
    count = 0
    while True:
        ex = r.read_example()
        if not ex:
            break
        # print('\n')
        # for lbl, data in ex.items():
        #     print('{}\t{}'.format(lbl, data))
        img_tensor, image = decode(ex)
        count += 1
    print('\n\nDecoded {} samples'.format(count))
    return img_tensor, image

In [ ]:
readem('gs://imagenet-jt/train/train-00000-of-01024')